In [1]:
# Title: DMRG for Bachelor Thesis
# Author: Aaron Sander
# Date: March 2020

# This program is used for initial learning of tensor network methods to be used
# in my bachelor thesis.
# It is an implementation of Matrix Product States (MPS) and Density Matrix
# Renormalization Group (DMRG) for finding the ground state of an arbitrary
# Hamiltonian

In [2]:
######################### IMPORTS ##############################################
import numpy as np

In [3]:
######################## CLASSES ##############################################
class MPO:
    def __init__(self, left_bound, inner, right_bound):
        # Leftmost lattice position
        self.left_bound = left_bound
        # Middle lattice positions
        self.inner = inner
        # Rightmost lattice position
        self.right_bound = right_bound

class MPS:
    def __init__(self, left_bound, inner, right_bound):
        self.left_bound = left_bound
        self.inner = inner
        self.right_bound = right_bound

In [4]:
###################### FUNCTIONS ##############################################
# Used to contract Hamiltonian horizontally from A->B
def contract_horizontal(A, B, pos, dir):
    if B.ndim == 4:  # Inner lattice positions
        if dir == 'right':
            tensor = np.einsum('ijk, iabc->ajbkc', A, B)
        if dir == 'left':  # Can be replaced with else, if for readability
            tensor = np.einsum('ijk, aibc->ajbkc', A, B)
        # Reshape collapses indices to (a, j*b, k*c) for i particles
        tensor = np.reshape(tensor, (3, 2**pos, 2**pos))

    if B.ndim == 3:  # Final lattice position
        if dir == 'right' or 'left':  # Seems to be direction-independent
            tensor = np.einsum('ijk, iab->jakb', A, B)
        # Reshape collapses indices to (j*a, k*b) for i particles
        tensor = np.reshape(tensor, (2**pos, 2**pos))

    return tensor

def contract_vertical(A, B, pos, dir):
    if B.ndim == 3:
        if dir == 'down':  # From Bra->Operator->Ket
            tensor = np.einsum('ij, abj->aib', A, B)
            tensor = np.reshape(tensor, (A.shape[0]*B.shape[0], B.shape[1]))
        if dir == 'up':  # From Ket->Operator->Bra
            tensor = np.einsum('ij, aib->ajb', A, B)
            tensor = np.reshape(tensor, (A.shape[1]*B.shape[0], B.shape[2]))
    if B.ndim == 2:
        if dir == 'down':
            tensor = np.einsum('ij, ja->ia', A, B)
            tensor = np.reshape(tensor, (A.shape[0]*B.shape[1]))
        if dir == 'up':
            tensor = np.einsum('ij, aj->ia', A, B)           
            tensor = np.reshape(tensor, (A.shape[0]*B.shape[0]))
    return tensor

# TODO: Verify correct reshape
# TODO: Verify if final lattice position is direction-independent
#       or if there is a mistake
# TODO: Determine if vertical/horizontal can be generalized and combined
# TODO: Generalize contract_horizontal
# TODO: Use tensordot for better readability?
# NOTE: In DMRG, the dimension of the final lattice position
#       will be a good marker for when to reverse direction
# NOTE: When collapsing with a wavefunction
#       the tensor shape will be based on size of
#       wavefunction matrices also, not just lattice pos

In [5]:
##################### INITIALIZATION ###########################################
# Operators
pauli_z = np.array([[1, 0],
                    [0, -1]])

pauli_x = np.array([[0, 1],
                    [1, 0]])

zero = np.zeros((2, 2))
identity = np.identity(2)
# Interaction parameter
g = 1
# Particles (Total lattice positions)
N = 3

# Initialization of Hamiltonian MPO (entries done by hand)
# Dimensions (1x3x2x2)->(3x2x2)
left_bound = np.array([identity, pauli_z, g*pauli_x])

# Dimensions (3x3x2x2)
inner = np.array([np.array([identity, pauli_z, g*pauli_x]),
                  np.array([zero, zero, pauli_z]),
                  np.array([zero, zero, np.identity(2)])])

# Dimensions (3x1x2x2)->3x2x2
right_bound = np.array([[g*pauli_x],
                        [pauli_z],
                        [identity]])
right_bound = np.squeeze(right_bound)  # Removes unnecessary index

MPO = MPO(left_bound, inner, right_bound)

In [6]:
##################### CONTRACT HAMILTONIAN L->R ###############################
# Initialize with first lattice position
tensor = MPO.left_bound
# Loop over all the inner lattice positions
for i in range(2, N):
    tensor = contract_horizontal(tensor, MPO.inner, i, "right")
# Final lattice position has different indices so is done alone
E_L = contract_horizontal(tensor, MPO.right_bound, N, "right")

##################### CONTRACT HAMILTONIAN L->R ###############################
# Initialize with first lattice position
tensor = MPO.right_bound
# Loop over all the inner lattice positions
for i in range(2, N):
    tensor = contract_horizontal(tensor, MPO.inner, i, "left")
# Final lattice position has different indices so is done alone
E_R = contract_horizontal(tensor, MPO.left_bound, N, "left")

E_L == E_R

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [7]:
# TODO: Verify MPS can be contracted left<->right
# We initialize each wavefunction matrix as:
# A_1_ket = (|+>    A_i_ket = 
#            |->)
d = 15

up_ket = np.zeros((d, 1))
up_ket[0,0] = 1
down_ket = np.zeros((d, 1))
down_ket[1,0] = 1

# Dimensions (2 x d)
A_1 = np.array([[up_ket],
                [down_ket]])
A_1 = np.squeeze(A_1)

# Dimensions (d x 2)
A_1_dagger = np.array(np.matrix(A_1).H)

MPS_bra = MPS(A_1_dagger, 0 , 0)
MPS_ket = MPS(A_1, 0, 0)

In [8]:
####################### CONTRACT FIRST LATTICE POSITION DOWNWARDS #################
# Dimensions (3*d x 2)
first_collapse = contract_vertical(MPS_bra.left_bound, MPO.left_bound, 1, 'down')
# Dimensions (3*d*d)
last_collapse_down = contract_vertical(first_collapse, MPS_ket.left_bound, 1, 'down')

####################### CONTRACT FIRST LATTICE POSITION UPWARDS ###################
# Dimensions (3*d x 2)
first_collapse = contract_vertical(MPS_ket.left_bound, MPO.left_bound, 1, 'up')
# Dimensions (3*d*d)
last_collapse_up = contract_vertical(first_collapse, MPS_bra.left_bound, 1, 'up')

last_collapse_down == last_collapse_up

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,